In [8]:
import requests
from bs4 import BeautifulSoup
import os
from multiprocessing.dummy import Pool
import json
from tqdm.notebook import tqdm
import time
import pandas as pd
import numpy as np
import re
from itertools import chain
pd.options.display.max_columns = 100

`check_links` - проверяет полноту парсинга ссылок на книги данного автора

In [9]:
def check_links(result, links):
    links_total = int(result.find('span', class_='rd-listing-count__total').get_text())
    return links_total == len(links)

In [10]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    for _ in range(n_attempts):
        try:
            response = requests.get(url, **kwargs)
            return response
        except:
            time.sleep(t_sleep)

`get_urls` - возвращает `tuple` из `id` автора и списка ссылок на его книги. Если список неполный, к `id` дописывает `_not_full`. Если сайт не отвечает, ссылка записывается в файл в `failed_pages`. Если не находит номер последней страницы, парсинг данного `id` останавливается и ссылка на последнюю страницу данного `id` записывается в файл в `failed_pages`.

In [11]:
def get_links(author):
    
    url = 'https://www.respublica.ru/authors/'
    page = 1
    links = []
    
    while True:
        link = url + author # + '?page=' + str(page)
        response = get_page(link, params={'page': page}, timeout=30)
        if not response:
            with open('failed_pages/{}.txt'.format(author), 'a') as file:
                print('No response from server\t', link + '?page=' + str(page), file=file)
            continue
        result = BeautifulSoup(response.content, 'html.parser')
        last_page = result.find_all('a', class_='rd-listing-pagination__link')
        if last_page:
            last_page = last_page[-1].get_text()
        else:
            with open('failed_pages/' + author + '.txt', 'a') as f:
                f.write('No last_page\t' + link + '\n')
            return (author, links)
        
        links_block = result.find_all('a',class_='rd-listing-product-item__image-wrapper', href=True)
        links += [link['href'] for link in links_block]
        
        if int(last_page) == page:
            if  not check_links(result, links):
                author += '_not_full'
            return (author, links)
        
        page += 1

Читаем список `id` авторов из файла и запускаем мультипроцессорный парсинг функцией `get_links`, преобразуем результат в словарь с ключами `id` авторов и значениями списками ссылок на книги.

In [14]:
with open('authors.txt', 'r') as f:
    authors = f.read().split('\n')
    
with Pool(processes=4) as pool:
    links = dict(tqdm(pool.imap(get_links, authors), total=len(authors)))
pool.join()

Проверяем полноту парсинга для всех авторов по флагу `_not_full`.

In [15]:
links_full = True
error_id = []
for item_link in links:
    if item_link.endswith('_not_full'):
        links_full = False
        error_id.append(item_link)
if links_full:
    print('All links are parsed')
else:
    print('Some links are not parsed')

All links are parsed


Сохраняем ссылки в файл.

In [16]:
with open('links.json', 'w') as f:
    json.dump(dict(links), f)

Загружаем ссылки из файла

In [17]:
with open('links.json', 'r') as f:
    links = json.load(f)

Функция `process_page` возращает словарь для карточки. Вспомогательные функции используются для парсинга отдельных блоков.

In [32]:
def parse_title(title_block):
    title = {}
    if title_block:
        title['Название'] = title_block.get_text()
    return title
    
def parse_author(author_block):
    author = {}
    if author_block:
        author['Автор'] = author_block.get_text()
    return author
    
def parse_preview(preveiw_block):
    preveiw = {}
    if preveiw_block:
        preveiw_item = preveiw_block.find('a', class_='download-pdf', href=True)
        if preveiw_item:
            preveiw['Превью'] = 'https://www.respublica.ru' + preveiw_item['href']
    return preveiw

def parse_img(img_block):
    img = {}
    if img_block:
        img['Изображение'] = 'https://www.respublica.ru' + img_block['src']
    return img

def parse_rating(rating_block):
    rating = {}
    if rating_block:
        value_block = rating_block.find('meta', content=True, itemprop='ratingValue')
        if value_block:
            rating['Оценка'] = value_block['content']
        reviewCount_block = rating_block.find('meta', content=True, itemprop='reviewCount')
        if reviewCount_block:
            rating['Число отзывов'] = reviewCount_block['content']
        ratingCount_block = rating_block.find('meta', content=True, itemprop='ratingCount')
        if ratingCount_block:
            rating['Число оценок'] = ratingCount_block['content']
    return rating

def parse_price(price_block):
    price = {}
    if price_block:
        sum_block = price_block.find('span', class_='num')
        if sum_block:
            price['Цена'] = ''.join(sum_block.get_text().split())
    return price

def parse_old_price(old_price_block):
    old_price = {}
    if old_price_block:
        old_sum_block = old_price_block.find('span', class_='prev')
        if old_sum_block:
            old_price['Цена (старая)'] = ''.join(old_sum_block.get_text().split()[0])
    return old_price

def parse_avail(avail_block):
    avail = {}
    if avail_block:
        avail['В наличии'] = bool(avail_block) and (avail_block.get_text() != 'Сообщить о поступлении')
    return avail

def parse_abstract(abstract_block):
    data = {}
    if abstract_block:
        blocks = abstract_block.find_all('p')
        abstract = ''
        for block in blocks:
            abstract += block.get_text()
        if not blocks:
            abstract = abstract_block.get_text()
        abstract = re.sub('\s+', ' ', abstract)
        data['Описание'] = abstract
    return data

def parse_table(props):
    table = {}
    for prop in props:
        prop_name = prop.find('span', itemprop='name')
        prop_value = prop.find('span', itemprop='value')
        if not prop_value:
            prop_value = prop.find('a', itemprop='value')
        if prop_name and prop_value:
            prop_name_string = prop_name.get_text()
            prop_value_string = prop_value.get_text()
            if prop_name_string and prop_value_string:
                table[prop_name_string] = prop_value_string
    return table

def parse_catrgory(category_block):
    data = {}
    if category_block:
        category = category_block.find_all('span', class_='rd-page-breadcrumbs-item')
        category = '; '.join(item.get_text() for item in category)
        data['Категория'] = category
    return data

def process_page(url):
    dn = 'https://www.respublica.ru'
    link = dn + url
    book = {'URL': link}

    response = get_page(link)
    if not response:
        return book
    result = BeautifulSoup(response.content, 'html.parser')
    content = result.find('div', class_='rd-page-product')
    
    book['ID'] = url.split('/')[-1].split('-')[0]
    category_block = content.find('div', class_='rd-page-breadcrumbs')
    category = parse_catrgory(category_block)
    book.update(category)
    
    main_info = content.find_all('div', class_='rd-page-product__row')
    if main_info and main_info[0]:
        left_fields = main_info[0].find('div', class_='rd-page-product__col-left')
        if left_fields:
            title_block = left_fields.find('h1', class_='rd-page-product__title')
            title = parse_title(title_block)
            book.update(title)
            
            author_block = left_fields.find('div', class_='rd-page-product__underline')
            author = parse_author(author_block)
            book.update(author)

            preveiw_block = left_fields.find('div', class_='pages-view')
            preveiw = parse_preview(preveiw_block)
            book.update(preveiw)

            img_block = left_fields.find('img', class_='rd-page-product__img', src=True)
            img = parse_img(img_block)
            book.update(img)

        right_block = main_info[0].find('div', class_='rd-page-product__col-right')
        if right_block:
            rating_block = right_block.find('span', itemprop='aggregateRating')
            rating = parse_rating(rating_block)
            book.update(rating)

            price_block = right_block.find('div', class_='rd-page-product__price')
            price = parse_price(price_block)
            book.update(price)

            old_price_block = right_block.find('div', class_='rd-page-product__price-old')
            old_price = parse_old_price(old_price_block)
            book.update(old_price)
            
            avail_block = right_block.find('span', class_='rd-page-product__buy-text')
            avail = parse_avail(avail_block)
            book.update(avail)

    if len(main_info) > 1:
        if main_info[1]:
            left_bottom_block = main_info[1].find('div', class_='rd-page-product__col-left')
            if left_bottom_block:
                abstract_block = left_bottom_block.find('div', class_='rd-page-product__desc-body')
                abstract = parse_abstract(abstract_block)
                book.update(abstract)

            right_bottom_block = main_info[1].find('div', class_='rd-page-product__desc-params')
            if right_bottom_block:
                props = right_bottom_block.find_all('p', class_='rd-page-product__desc-param')
                table = parse_table(props)
                book.update(table)
                

    return book

Создаем список всех ссылок и запускаем мультипроцессорный парсинг. Полученный список словарей преобразуем в `pandas.DataFrame`

In [18]:
urls = list(chain.from_iterable(links.values()))

In [19]:
len(urls)

2453

In [35]:
with Pool(processes=4) as pool:
    result = list(tqdm(pool.imap(process_page, urls), total=len(urls)))
pool.join()
df = pd.DataFrame(result)

Посмотрим на полученный датафрейм

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 41 columns):
URL                       2461 non-null object
ID                        2461 non-null object
Категория                 2461 non-null object
Название                  2461 non-null object
Автор                     2461 non-null object
Превью                    1182 non-null object
Изображение               2461 non-null object
Цена                      2461 non-null object
В наличии                 2461 non-null bool
Описание                  2461 non-null object
ISBN                      2146 non-null object
Издательство              2259 non-null object
Серия                     1656 non-null object
Обложка                   2252 non-null object
Формат                    2442 non-null object
Количество страниц        2250 non-null object
Год издания               2238 non-null object
Язык                      2241 non-null object
Раздел                    1848 non-null object
На

По всем ссылкам была получена информация. Возьмем 5 рандомных книг и посмотрим на данные по ним.

In [38]:
df_sample = df.loc[np.random.randint(0, df.shape[0], 5)]

In [39]:
df_sample

,URL,ID,Категория,Название,Автор,Превью,Изображение,Цена,В наличии,Описание,ISBN,Издательство,Серия,Обложка,Формат,Количество страниц,Год издания,Язык,Раздел,Направление,Страна-производитель,"Вес, г",Возрастные ограничения,Оценка,Число отзывов,Число оценок,Цена (старая),Жанр,Герои,Эпоха,Возраст,Иллюстратор,Тип,Вид бумаги,Материал,"Размер, см",Пол,Назначение,Тематика,Упаковка,Рисунок
5,https://www.respublica.ru/knigi/hudozhestvenna...,484943,Книги; Художественная литература; Фантастик...,"Девочка, которая любила Тома Гордона",Стивен Кинг,https://www.respublica.ru/items/360505/downloa...,https://www.respublica.ru/uploads/01/00/00/6o/...,170,True,"Девятилетняя Триша заблудилась в лесу, и чем д...",978-5-17-111310-0,АСТ,Современная зарубежная проза,Мягкая,12 х 18,288,2018,Русский,Зарубежные,Фэнтези,Россия,150,0+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791,https://www.respublica.ru/knigi/hudozhestvenna...,517012,Книги; Художественная литература; Драматургия,Великий Гэтсби. The Great Gatsby,Фрэнсис Скотт Фицджеральд,https://www.respublica.ru/items/376944/downloa...,https://www.respublica.ru/uploads/01/00/00/c6/...,235,False,"""Великий Гэтсби"" — самый известный роман Фрэнс...",978-5-04-095999-0,Эксмо,Билингва Bestseller,Мягкая,"18 x 11,5",448,2018,Несколько языков,Зарубежные,NaN,Россия,276,16+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2104,https://www.respublica.ru/knigi/hudozhestvenna...,444931,Книги; Художественная литература; Детективы,Десять негритят. Изумруд раджи,Кристи Агата,https://www.respublica.ru/items/320238/downloa...,https://www.respublica.ru/uploads/00/00/00/6f/...,570,False,Десять негритят:Десять никак не связанных межд...,978-5-699-92329-8,Эксмо,Агата Кристи. Золотая коллекция,Твердая,13 х 21,416,2016,Русский,Зарубежные,Криминальные,Россия,463,16+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2430,https://www.respublica.ru/knigi/detskie-knigi/...,528782,Книги; Детские книги; Художественная литера...,Приключения суперсыщика Калле Блумквиста,Астрид Линдгрен,NaN,https://www.respublica.ru/uploads/00/00/00/cy/...,800,False,Кто из мальчишек не мечтал в детстве стать суп...,978-5-389-17428-3,Махаон,Книги Астрид Линдгрен,Твердая,21 х 29,384,2019,Русский,Зарубежные,NaN,Россия,NaN,NaN,NaN,NaN,NaN,920,NaN,NaN,NaN,3+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,https://www.respublica.ru/knigi/hudozhestvenna...,061685,Книги; Художественная литература; Детективы,Нефритовые четки,Борис Акунин,NaN,https://www.respublica.ru/uploads/00/00/00/1f/...,680,True,Последний раз мы встречались с Эрастом Петрови...,978-5-8159-1050-8,Захаров,NaN,Твердая,13 х 20,704,2010,Русский,Отечественные,Исторические,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Сохраняем датафрейм в `.csv` файл

In [41]:
df.sort_values(by=['ID'], inplace=True)
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)